# Prototyping:
1. Generate new answers for existing exercises
2. Content scaffolding (generate first article draft)
3. Simplify language and provide different difficulty versions for students

## Task 3
* [Experiment with prompting](https://docs.google.com/presentation/d/1wImIku229N92JU-yXZa1NO9lAqwtM5_jY4mmLI-fl_U/edit#slide=id.g256f714229b_1_116)
  * Find suitable prompts (x)
  * Create examples by hand to improve prompts 

* Connect to database (x)
* Extract content -> only text content (x) => Improve output
* Send content to ChatGPT


In [13]:
import os
import openai
import dotenv

In [15]:
env_file = 'OPENAI.env'
dotenv.load_dotenv(env_file, override=True)
openai.organization = os.environ.get("OPENAI_ORGANIZATION_ID")
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [4]:
import json
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo",
    charset="latin1"
)

def query(sql):
    c = db.cursor()
    c.execute(sql)
    
    return c.fetchall()

def getCurrentEntityRevision(current_revision_id):
    results = query("""
        select
            max(entity_revision.id),
            max(entity.id),
            max(type.name),
            json_objectagg(entity_revision_field.field, entity_revision_field.value) as fields
        from entity
            join type on entity.type_id = type.id
            join entity_revision on entity_revision.id = entity.current_revision_id
            join entity_revision_field on entity_revision_field.entity_revision_id = entity_revision.id
        where entity.id = {}
        group by entity_revision.id
    """.format(current_revision_id))

    return [{
        "currentRevisionId": result[0],
        "entityId": result[1],
        "type": result[2],
        **json.loads(result[3])
    } for result in results]

[{'currentRevisionId': 264292,
  'entityId': 1795,
  'type': 'article',
  'title': 'Ableitung',
  'changes': 'Rechtschreibung korrigiert',
  'content': '{"plugin":"article","state":{"introduction":{"plugin":"articleIntroduction","state":{"explanation":{"plugin":"text","state":[{"type":"p","children":[{"text":"Die "},{"text":"Ableitung","strong":true},{"text":" einer Funktion "},{"type":"math","src":"f","inline":true,"children":[{"text":"f"}]},{"text":" an einer Stelle "},{"type":"math","src":"x","inline":true,"children":[{"text":"x"}]},{"text":" gibt die "},{"text":"Steigung des Graphen der Funktion","strong":true},{"text":" an dieser Stelle an."}]},{"type":"p","children":[{"text":"Bezeichnet wird sie zumeist mit "},{"type":"math","src":"f\'(x)","inline":true,"children":[{"text":"f\'(x)"}]},{"text":"."}]}]},"multimedia":{"plugin":"image","state":{"src":"https://assets.serlo.org/6227206de64da_69fce42ecb2dedad112be172f4e98b90d71456b5.svg","caption":{"plugin":"text","state":[{"type":"p","

In [21]:
# TODO: remove latex and restore umlaute
def extract_strings_from_object(obj):
    strings = []

    if isinstance(obj, dict):
        for key, value in obj.items():
            if key == 'text':
                strings.append(value)
            else:
                strings.extend(extract_strings_from_object(value))
    elif isinstance(obj, list):
        for item in obj:
            strings.extend(extract_strings_from_object(item))

    return strings


content = json.loads(getCurrentEntityRevision(1795)[0]["content"])
text = " ".join(extract_strings_from_object(content))


In [16]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", 
     "content": "Erkläre den Begriff der Ableitung einem 10-Jährigem und einem 16-Jährigen."}
  ]
)

In [17]:
print(completion.choices[0].message.content)

Ein 10-Jähriger könnte die Ableitung als "eine Methode, um herauszufinden, wie schnell sich etwas verändert" erklärt bekommen. Beispielweise könnte man sagen: "Stell dir vor, du fährst Fahrrad und willst wissen, wie schnell du dich bewegst. Du könntest die Ableitung benutzen, um herauszufinden, wie schnell du dich von einem Punkt zum anderen bewegst. Das hilft dir, zu wissen, ob du schnell fährst oder langsamer wirst."

Ein 16-Jähriger könnte die Ableitung als "die Steigung einer Funktion an einem bestimmten Punkt" erklärt bekommen. Beispielweise könnte man sagen: "Stell dir vor, du hast eine Funktion, die den Weg beschreibt, den du mit deinem Auto fährst. Die Ableitung dieser Funktion würde dir sagen, wie schnell du dich an einer bestimmten Stelle bewegst. Wenn die Ableitung positiv ist, fährst du schneller, wenn sie negativ ist, verlangsamst du dich. Die Ableitung hilft dir also, die Geschwindigkeit deiner Bewegung zu verstehen."


In [22]:
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", 
     "content": "Erstelle aus diesem Text eine Version für einem 10-Jährigem und einem 16-Jährigen: {}".format(text)}
  ]
)
print(completion.choices[0].message.content)

Für einen 10-Jährigen:

Die Ableitung einer Funktion an einer bestimmten Stelle sagt uns wie steil der Graph der Funktion an dieser Stelle ist. Wenn die Ableitung größer als null ist, steigt der Graph an dieser Stelle. Wenn die Ableitung kleiner als null ist, fällt der Graph. An den extremsten Stellen und an Terrassenpunkten ist die Ableitung gleich null. Die Ableitung hilft uns dabei, die Extremstellen und die Monotonie der Funktion zu bestimmen. Funktionen, die an jeder Stelle eine Ableitung haben, nennt man differenzierbar. Das Berechnen der Ableitung nennt man Differenzieren. Es gibt Regeln, mit denen man die Ableitung berechnen kann.

Für einen 16-Jährigen:

Die Ableitung einer Funktion an einer bestimmten Stelle gibt uns die Steigung des Graphen dieser Funktion an. Wenn die Ableitung an einem Punkt größer als null ist, steigt der Graph an dieser Stelle. Ist die Ableitung kleiner als null, fällt der Graph an dieser Stelle. An den Stellen, an denen die Ableitung gleich null ist, ha